Step 1: Install PySpark

In [1]:
#Install PySpark
!pip install pyspark

Step 2 : Import PySpark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, hour, dayofweek

Step 3 : Intialization

In [3]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Big Data Analysis - NYC Taxi Trip") \
    .getOrCreate()

Step 4 : Download the Dataset

In [4]:
# Download sample NYC Yellow Taxi dataset (January 2023)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O yellow_tripdata_2023-01.parquet


--2025-06-21 14:02:43--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.163.157.7, 3.163.157.133, 3.163.157.96, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.163.157.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   230MB/s    in 0.2s    

2025-06-21 14:02:43 (230 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



Step 5 : Load the Dataset

In [5]:

file_path = "yellow_tripdata_2023-01.parquet"
df = spark.read.parquet(file_path)  # ✅ Make sure df is defined here

# Show the schema to confirm it loaded
df.printSchema()


root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



Step 6 : Data Cleaning

In [6]:
df_clean = df.filter(
    (col("trip_distance") > 0) &
    (col("fare_amount") > 0) &
    (col("passenger_count") > 0)
)

Step 7 : Feature Engineering

In [7]:
df_features = df_clean.withColumn("pickup_hour", hour("tpep_pickup_datetime")) \
      .withColumn("pickup_day", dayofweek("tpep_pickup_datetime"))

Step 8 : Perform Analysis

In [8]:
# 1. Average fare by hour
avg_fare_by_hour = df_features.groupBy("pickup_hour").agg(avg("fare_amount").alias("avg_fare")).orderBy("pickup_hour")

# 2. Most popular pickup hours
popular_hours = df_features.groupBy("pickup_hour").agg(count("*").alias("trip_count")).orderBy(col("trip_count").desc())

# 3. Average trip distance by day of the week
avg_distance_by_day = df_features.groupBy("pickup_day").agg(avg("trip_distance").alias("avg_distance")).orderBy("pickup_day")

Step 9 : Display the Results

In [9]:
print("🚕 Average fare by hour:")
avg_fare_by_hour.show()

print("⏰ Most popular pickup hours:")
popular_hours.show()

print("📅 Average trip distance by day of week:")
avg_distance_by_day.show()

🚕 Average fare by hour:
+-----------+------------------+
|pickup_hour|          avg_fare|
+-----------+------------------+
|          0|19.802777784760078|
|          1| 17.82256348420773|
|          2|16.722530014716988|
|          3| 17.73772003352903|
|          4|22.253980171760563|
|          5|26.456131830008786|
|          6|22.150058079859775|
|          7|18.919863108407178|
|          8|17.425238969055147|
|          9| 17.60772748768832|
|         10|17.726938463244128|
|         11|17.407977516487446|
|         12| 17.75597548569939|
|         13|18.462829446340645|
|         14| 19.76365945811615|
|         15|19.290242855602155|
|         16|19.539086710022136|
|         17|18.626508726030057|
|         18|16.977947860339086|
|         19|17.646060443500723|
+-----------+------------------+
only showing top 20 rows

⏰ Most popular pickup hours:
+-----------+----------+
|pickup_hour|trip_count|
+-----------+----------+
|         18|    203607|
|         17|    197226|
|   

Step 10 : Stop the Spark Session

In [10]:
spark.stop()